# Vadere EMA connector demo
For more information on the use of the EMA Workbench, please refer to the [official EMA Workbench documentation](https://emaworkbench.readthedocs.io/en/latest/). This demo is based on the code provided on the documentation pages.

## Step 1: imports
The first step is to import the needed modules. This depends on the use and the type of analysis that is intended. The most important one here is the VadereModel from the model connectors. As said, please refer for more information on this to the [official EMA Workbench documentation](https://emaworkbench.readthedocs.io/en/latest/).

In [1]:
from ema_workbench import (perform_experiments, RealParameter, ema_logging,
                           CategoricalParameter, MultiprocessingEvaluator,
                           ScalarOutcome, IntegerParameter, RealParameter)
from ema_workbench.em_framework.parameters import Category
from ema_workbench.connectors.vadere import VadereModel
import pandas as pd
import numpy as np

/home/tevito/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/connectors/__init__.py:17: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
/home/tevito/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/connectors/__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)


## Step 2: Setting up the model
In this example, we use the Vadere model from this research.  

In [2]:
# This model saves scalar results to a density.txt and speed.txt file.
model = VadereModel('model', 
                    vadere_jar='vadere-console.jar',
                    processor_files=[
                        'density.txt',
                        'speed.txt'
                    ],
                    model_file='baseCaseData.scenario',
                    wd='/home/tevito/Documents/EPA/Year2/thesis/thesis-drive/model/connector/output')

In [3]:
# set the number of replications to handle model stochasticity
model.replications = 2

Note that for specifying model uncertainties (and potential levers), the Vadere model class can change any variable present in the model file (Vadere scenario). To realise this, a exact location to the variable of interest in the Vadere scenario file has to be specified. Vadere scenario files follow a nested dictionary structure. Therefore, the exact location of the variable should be passed in a list of argumentes, passed as one string. See the example below, that variates the spawnNumber and maxSpawnNumber of source 0 and source 1 in the Vadere model.

![alt text](parameters.png "Parameters")

To reduce complexity, this example demonstrates the specification of three parameter uncertainties (instead of the 10 illustrated above)

In [4]:
model.uncertainties = [
    IntegerParameter(
        name='spawnFrequencyA',
        lower_bound=1,
        upper_bound=5,
        variable_name=[
            '("scenario", "topography", "sources", 0, "distributionParameters", "updateFrequency")',
        ]
    ),
    RealParameter(
        name='μFreeFlowSpeed',
        lower_bound=0.66,
        upper_bound=1.16,
        variable_name=[
            '("scenario", "topography", "attributesPedestrian", "speedDistributionMean")',
        ]
    ),
    RealParameter(
        name='pedPotentialHeight',
        lower_bound=5.0,
        upper_bound=50.0,
        variable_name=[
            '("scenario", "attributesModel", "org.vadere.state.attributes.models.AttributesPotentialCompactSoftshell", "pedPotentialHeight")',
        ]
    )
]

The model outcomes can be specified by passing the exact name as present in the output file (speed.txt here). The naming convention depends on the used Vadere data processors, but usually follows the name + id of the processor. When in doubt, it is advised to do a demo Vadere run using the Vadere software and to inspect the generated output files. Note that we take the mean of the outcomes here, since we specified multiple replications.

In [5]:
model.outcomes = [
    ScalarOutcome(
        name='meanSpeed',
        variable_name='mean_area_speed_processor-PID6',
        function=np.mean
    ),
    ScalarOutcome(
        name='meanDensity',
        variable_name='mean_density_counting_normed_processor-PID2',
        function=np.mean
    ),
    ScalarOutcome(
        name='maxDensity',
        variable_name='max_density_counting_normed_processor-PID7',
        function=np.mean
    ),
]

## Step 3: Performing experiments
The last step is to perform experiment with the Vadere model. Both sequential runs as runs in parallel are supported. Note however that a Vadere run can use a lot of RAM, and using all available CPU cores can lead to performance issues in some cases. 

In [6]:
# enable EMA logging
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [7]:
# run in sequential 2 experiments
results_sequential = perform_experiments(
    model,
    scenarios=2,
    uncertainty_sampling='lhs'
)

[MainProcess/INFO] performing 2 scenarios * 1 policies * 1 model(s) = 2 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] experiments finished


In [8]:
# run 8 experiments in parallel
with MultiprocessingEvaluator(model, n_processes=8) as evaluator:
        experiments, outcomes = evaluator.perform_experiments(
                scenarios=8,
                uncertainty_sampling='lhs'
)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 8 scenarios * 1 policies * 1 model(s) = 8 experiments
[ForkPoolWorker-7/INFO] finalizing
[ForkPoolWorker-8/INFO] finalizing
[ForkPoolWorker-3/INFO] finalizing
[ForkPoolWorker-1/INFO] finalizing
Process ForkPoolWorker-7:
Traceback (most recent call last):
  File "/home/tevito/anaconda3/envs/CrowdSim/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tevito/anaconda3/envs/CrowdSim/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tevito/anaconda3/envs/CrowdSim/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/tevito/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/em_framework/ema_multiprocessing.py", line 168, in worker
    return experiment, experiment_runner.run_experiment(experiment)
  File "/h

KeyboardInterrupt: 

## Inspect the results
we can now look at the results and plot them. For more extensive exploratory analysis methods, or direct search, please refer to the [official EMA Workbench documentation](https://emaworkbench.readthedocs.io/en/latest/).

In [ ]:
experiments.head()

In [ ]:
pd.DataFrame(outcomes).head()

In [ ]:
from ema_workbench.analysis import pairs_plotting

fig, axes = pairs_plotting.pairs_scatter(experiments, outcomes, group_by='policy',
                                        legend=False)
fig.set_size_inches(8,8)
plt.show()